# MDPs, Trajetórias e Retornos


In [51]:
import gym
import time
import numpy as np

# vamos focar nesses três ambientes por serem mais simples
#env = gym.make("MountainCar-v0")
#env = gym.make("CartPole-v1")
env = gym.make("Taxi-v3")

![Figura mostrando interação agente(política)-ambiente](mdp.png "Interação agente - ambiente")

# 1. Episódio e Trajetória

Um *episódio* é uma execução da tarefa (ou do ambiente gym). 

E a *trajetória* é a sequência de estados (observações), ações e recompensas do episódio. Assumindo um episódio de $T$ passos (ações aplicadas):

$S_0 \rightarrow A_0 \rightarrow R_1 \rightarrow S_1 \rightarrow A_1 \rightarrow R_2 \rightarrow S_2 \rightarrow \cdots S_{T-1} \rightarrow A_{T-1} \rightarrow R_T \rightarrow S_T$

Vamos ilustrar um episódio em um MDP usando o ambiente *"env"* escolhido no código acima. 

Estamos assumindo que o episódio encerrou de fato (chegou em um estado final) em *$n$="TOTAL_STEPS"* passos.

In [4]:
TOTAL_STEPS = 5

i = 0
obs = env.reset()
print(f"S0 = {obs}")

done = False

# roda apenas alguns passos
for i in range(0,TOTAL_STEPS):
    #env.render()
    action = env.action_space.sample()
    print(f" A{i} = {action}")

    next_obs, reward, done, info = env.step(action)

    print(f"  R{i+1} = {reward}")
    print(f"S{i+1} = {next_obs}")

    obs = next_obs
    time.sleep(0.1)

env.close()

S0 = 111
 A0 = 1
  R1 = -1
S1 = 11
 A1 = 1
  R2 = -1
S2 = 11
 A2 = 4
  R3 = -10
S3 = 11
 A3 = 2
  R4 = -1
S4 = 31
 A4 = 3
  R5 = -1
S5 = 11


Os detalhes do *episódio* que mostramos acima são chamamos de *trajectory* ou *rollout*.

Dependendo do algoritmo, vamos precisar analisar essas informações em trios (S,A,R) ou quádruplas (S,A,R,S) ou até quíntuplas (S,A,R,S',A').

Abaixo, vamos agrupar e guardar em trio, para preparar para o algoritmo Monte Carlo.

In [ ]:
# pular?

# 2.Calcular os Retornos

O *retorno (final)* $G$ é uma medida da recompensa total obtida ao longo de um episódio. 

Em um MDP, o objetivo é otimizar o valor médio de $G$, para infinitos episódios.

Para isso, vamos assumir a política abaixo, que escolhe a ação $0$ com 50% de probabilidade, ou outra ação, caso contrário.


In [45]:
num_actions = env.action_space.n

def policy(state):
    x = np.random.random()
    if x <= 0.5:
        return 0
    else:
        return np.random.randint(1, num_actions)


### 2.1 Retorno final do episódio ($G$)



Para um episódio com $n$ passos, o *retorno* (não-descontado) é calculado assim:

$ G = R_1 + R_2 + R_3 + \cdots + R_n = \displaystyle\sum_{i=1}^{n} R_i$

O mais usado é o *retorno descontado* de um episódio.

Neste caso, $G$ é uma soma que "atenua" recompensas mais distantes, valorizando mais as recompensas iniciais. (Você prefere receber 100 reais agora, de uma vez, ou em 100 parcelas de 1 real?)

Para isso, a cada passo, a recompensa tem uma redução dada por um parâmetro $\gamma\;$, tal que $0 < \gamma \leq 1$.

Para um episódio com $n$ passos, o *retorno descontado* é calculado assim:

$ G = R_1 + \gamma R_2 + \gamma^2 R_3 + \cdots + \gamma^{(n-1)} R_n = \displaystyle\sum_{i=1}^{n} R_i$

In [8]:
# deixo como exercício
# raramento vamos calcular diretamente, durante a execução

### 2.2 Retornos intermediários a cada passo ($G_i$)



Também podemos calcular um retorno parcial, de um certo episódio, a partir de um passo específico $i$ do episódio:

- $ G_0 = R_1 + \gamma R_2 + \gamma^2 R_3 + \gamma^3 R_4 + \gamma^4 R_5 \cdots \;\;= G$ 
- $ G_1 = \;\;\;\;\;\;\;\;\;  R_2 + \;\gamma R_3 + \gamma^2 R_4 + \cdots $ 
- $ G_2 = \;\;\;\;\;\;\;\;\; \;\;\;\;\;\;\;\;\;\; R_3 + \;\gamma R_4 + \cdots $ 
- $\cdots$ 
- $ G_n = 0 $

Propriedade:
- $ G_{i-1} = r_i + \gamma G_i $

# 3. Funções de Valor

### 3.1 Função de valor do estado V(s)

Calcula o retorno esperado a partir de todo estado $s$.

In [ ]:
# fica como exercício

### 3.2 Função de valor do estado-ação Q(s,a)

Calcula o retorno esperado a partir do par $(s,a)$.

*"Quando estava no estado $s$ e fez a ação $a$, qual a recompensa futura esperada?"*

In [48]:
# vamos fazer?

# 4. Introdução aos Métodos Baseados em Q-table

Suponha que você partiu de uma política qualquer (provavelmente ruim) e calculou o *Q* dessa política.

De que forma poderíamos melhorá-la?